In [116]:
import pandas as pd
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
from openai import OpenAI

In [117]:

df = pd.read_csv('RAG_df2.csv', skipinitialspace=True, engine='python', error_bad_lines=False, encoding= "UTF-8")
df.head(2)

C:\Users\lol_a\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,title,author,year,isbn,description,pages,genres,rating
0,"Reyes Malditos I, Los. El Rey de Hierro",Maurice Druon,2003,8466612041,The terrible cursion of the Templars. B! All d...,296,Fiction,4.0
1,El Ocho,Katherine Neville,2004,8466616160,The valuable pieces of the Montglane chess gam...,860,Fiction,4.0


In [119]:
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [120]:
def combine_fields(doc):
    return (
        f"Título: {doc['title']}. "
        f"Autor: {doc['author']}. "
        f"Año: {doc['year']}. "
        f"Descripción: {doc['description']}. "
        f"Páginas: {doc['pages']}. "
        f"Géneros: {doc['genres']}. "
        f"Rating: {doc['rating']}."
    )

In [121]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

qdrant.recreate_collection(
    collection_name = "books",
    vectors_config = models.VectorParams(
        size = encoder.get_sentence_embedding_dimension(),
        distance = models.Distance.COSINE
    )
)
# vectorize!
qdrant.upload_points(
    collection_name="books",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(combine_fields(doc)).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

c:\Users\lol_a\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\lol_a\AppData\Local\Temp\ipykernel_25812\320348425.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


In [122]:
user_prompt = "Give me a very bad fantasy novel with a very bad rating"

In [123]:

hits = qdrant.search(
    collection_name="books",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'title': 'Dark Lord of Derkholm', 'author': 'Diana Wynne Jones', 'year': 1998, 'isbn': '0688160042', 'description': 'Mr. Chesney operates Pilgrim Parties, a tour group that takes paying participants into an outer realm where the inhabitants play frightening and foreboding roles. The time has come to end the staged madness . . . but can it really be stopped? Master storyteller Diana Wynne Jones serves up twists and turns, introduces Querida, Derk, Blade, and Shona and a remarkable cast of wizards, soldiers, kings, dragons, and griffins, and mixes in a lively dash of humor. With all the ingredients of high fantasy, this unforgettable novel will delight fans old and new.', 'pages': 532, 'genres': 'Juvenile Fiction', 'rating': 3.5} score: 0.425179933608257
{'title': 'The Waste Lands (The Dark Tower, Book 3)', 'author': 'Stephen King', 'year': 2003, 'isbn': '0452284716', 'description': 'Roland, the Last Gunslinger, and his companions--Eddie Dean and Susannah--cross the desert of damnation,

In [124]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [125]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-local"
)
completion = client.chat.completions.create(
    model="Llama-3.2-3B-Instruct.Q6_K.gguf",
    messages=[
        {"role": "system", "content": "Assistant is a chatbot that helps you find a good book recommendation."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='It looks like I provided some actual book information instead of a bad fantasy novel with a low rating.\n\nLet me try again. Here\'s a fictional example of a bad fantasy novel with a very low rating:\n\n**"The Dragon\'s Bane" by Balthazar McSnazz**\n\nRating: 1.8/5 stars\n\nThis epic fantasy novel follows the journey of a clumsy wizard named Eryndor Thorne as he attempts to defeat the dark lord Xarxos and save the world from certain doom. Along the way, Eryndor must navigate treacherous landscapes, battle hordes of orcs, and deal with his annoyingly chatty sidekick, a talking chicken named Cluck Norris.\n\nThe book is filled with cliches, predictable plot twists, and cringe-worthy dialogue. The characters are one-dimensional and lack any real depth or development. The world-building is also lacking, with a convoluted and confusing magic system that will leave readers scratching their heads.\n\nDespite its many flaws, "The Dragon\'s Bane" has gained a cult